In [4]:
import pandas as pd
import world_trade_data as wits
import requests

In [5]:
wits.get_countries()

,name,notes,countrycode,isreporter,ispartner,isgroup,grouptype
iso3Code,,,,,,,
AFG,Afghanistan,,004,False,False,No,N/A
ALB,Albania,,008,False,False,No,N/A
DZA,Algeria,,012,False,False,No,N/A
ASM,American Samoa,,016,True,False,No,N/A
AND,Andorra,,020,False,False,No,N/A
...,...,...,...,...,...,...,...
YEM,Yemen,,887,False,False,No,N/A
YDR,Yemen Democratic,,720,True,False,No,N/A
YUG,"Yugoslavia,FR(Serbia/Montenegr",,890,True,False,No,N/A


In [6]:
wits.get_products()

,isgroup,nomenclaturecode,grouptype,productdescription,notes
productcode,,,,,
999999,N/A,N/A,N/A,Not Applicable,
01-05_Animal,Yes,H0,Sector,Animal,
06-15_Vegetable,Yes,H0,Sector,Vegetable,
16-24_FoodProd,Yes,H0,Sector,Food Products,
25-26_Minerals,Yes,H0,Sector,Minerals,
27-27_Fuels,Yes,H0,Sector,Fuels,
28-38_Chemicals,Yes,H0,Sector,Chemicals,
39-40_PlastiRub,Yes,H0,Sector,Plastic or Rubber,
41-43_HidesSkin,Yes,H0,Sector,Hides and Skins,
